# Neural Particles 2D

## 2D Dataset

### Setup

In [1]:
import os
from subprocess import Popen, PIPE

dir_loc = "2D_SPH/"

data_version = 0

def run_manta(scene, param={}):
    command = [dir_loc+"build/manta", dir_loc+scene]

    for k, v in param.items():
        command += [k, str(v)]
        
    print(" ".join(command) + "\n")

    proc = Popen(command, stdin=None, stdout=PIPE, stderr=PIPE)

    for line in proc.stdout:
        print(line.decode('utf-8'))
    for line in proc.stderr:
        print(line.decode('utf-8'))
    

### Generate High-res Data

In [19]:
highres_prefix = ("2D_data/highres/drop_2D_v%02d" % data_version) + "_%03d"

frame_count = 50
sub_res = 2
res = 150

param = {}

#disable gui
param['gui'] = 1

# resolution of domain
param['res'] = 150
param['sres'] = 2

# output file format
param['out'] = highres_prefix

# write only every 30th frame -> 30 frames are one timestep
param['fps'] = 30

# simulation time (how many frames)
param['t_end'] = float(frame_count) / param['fps']

run_manta("scenes/2D_drop.py", param)

2D_SPH/build/manta 2D_SPH/scenes/2D_drop.py fps 30 out 2D_data/highres/drop_2D_v00_%03d gui 1 t_end 1.6666666666666667 sres 2 res 150

Version: mantaflow mac 64bit fp1 commit ae47b535eddcda2611e983fcdbff644e0c817d9e from Oct 18 2017, 21:36:37

Loading script '2D_SPH/scenes/2D_drop.py'

Added & initialized 9344 particles

Added & initialized 2928 particles

   Frame 0, dt: 0.0006420951, 0/0.03333334 lock:0

   Frame 0, dt: 0.0006420951, 0.0006420951/0.03333334 lock:0

writing particles pp to uni file 2D_data/highres/drop_2D_v00_000_ps.uni

writing particle data pT to uni file 2D_data/highres/drop_2D_v00_000_pt.uni

writing particle data pV to uni file 2D_data/highres/drop_2D_v00_000_pv.uni

writing particle data pD to uni file 2D_data/highres/drop_2D_v00_000_pd.uni

writing particle data pP to uni file 2D_data/highres/drop_2D_v00_000_pP.uni

Writing grid  to uni file 2D_data/highres/drop_2D_v00_000_sdf.uni

Writing grid  to uni file 2D_data/highres/drop_2D_v00_000_vel.uni

Writing grid 

### Generate Low-res Data

In [22]:
lowres_prefix = ("2D_data/lowres/l_drop_2D_v%02d" % data_version) + "_%03d"

param = {}
param['res'] = 15
param['factor'] = 10
param['gui'] = 1
param['in'] = highres_prefix
param['out'] = lowres_prefix
param['pause'] = 0
param['t'] = frame_count

run_manta("scenes/tools/downsample.py", param)

2D_SPH/build/manta 2D_SPH/scenes/tools/downsample.py pause 0 out 2D_data/lowres/l_drop_2D_v00_%03d factor 10 t 50 in 2D_data/highres/drop_2D_v00_%03d gui 1 res 15

Version: mantaflow mac 64bit fp1 commit ae47b535eddcda2611e983fcdbff644e0c817d9e from Oct 18 2017, 21:36:37

Loading script '2D_SPH/scenes/tools/downsample.py'

reading particles pp from uni file 2D_data/highres/drop_2D_v00_000_ps.uni

reading particle data pT from uni file 2D_data/highres/drop_2D_v00_000_pt.uni

reading particle data pV from uni file 2D_data/highres/drop_2D_v00_000_pv.uni

reading particle data pD from uni file 2D_data/highres/drop_2D_v00_000_pd.uni

reading particle data pP from uni file 2D_data/highres/drop_2D_v00_000_pp.uni

 Deleted 11007 particles

writing particles pp to uni file 2D_data/lowres/l_drop_2D_v00_000_ps.uni

Writing grid  to uni file 2D_data/lowres/l_drop_2D_v00_000_sdf.uni

Writing grid  to uni file 2D_data/lowres/l_drop_2D_v00_000_vel.uni

Writing grid  to uni file 2D_data/lowres/l_drop_

## Train Neural Network

### Setup Neural Network

In [4]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation

conv_f = 3
conv_k = 3
conv_s = (1, 1)

deconv_f = 3
deconv_k = 3
deconv_s = (1, 1)

input_shape = (64, 64, 1)

model = Sequential()

model.add(keras.layers.Conv2D(input_shape=input_shape, filters=conv_f,            \
                              kernel_size=conv_k, strides=conv_s,                 \
                              padding='valid', data_format=None,                  \
                              dilation_rate=(1, 1), activation=None,              \
                              use_bias=True, kernel_initializer='glorot_uniform', \
                              bias_initializer='zeros', kernel_regularizer=None,  \
                              bias_regularizer=None, activity_regularizer=None,   \
                              kernel_constraint=None, bias_constraint=None))

model.add(Dense(units=100))
model.add(Activation('relu'))

model.add(keras.layers.Conv2DTranspose(filters=deconv_f, kernel_size=deconv_k,    \
                                       strides=deconv_s, padding='valid',         \
                                       data_format=None, activation=None,         \
                                       use_bias=True,                             \
                                       kernel_initializer='glorot_uniform',       \
                                       bias_initializer='zeros',                  \
                                       kernel_regularizer=None,                   \
                                       bias_regularizer=None,                     \
                                       activity_regularizer=None,                 \
                                       kernel_constraint=None,                    \
                                       bias_constraint=None))

Using TensorFlow backend.
